# Python to SQL, and back again
In this codealong we will show you how to create a relational database from your pandas DataFrames.
> **To run this notebook you will need to work locally and not on colab.**

---
## 1.&nbsp; Import libraries 💾
If you haven't already installed sqlalchemy, you will need to. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [ ]:
# install if needed
!pip install sqlalchemy
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 804.3 kB/s eta 0:00:00


In [1]:
import pandas as pd

---
## 2.&nbsp; Relational Databases 📂

Creating DataFrames in python and pandas often results in tables with repeated information, as shown in the example below.
<br>

| author_name | book_title | year_published |
| --- | --- | --- |
| Arthur Conan Doyle | The Adventures of Sherlock Holmes | 1887 |
| J.R.R. Tolkien | The Hobbit | 1937 |
| J.R.R. Tolkien | The Lord of the Rings | 1954 |
| Harper Lee | To Kill a Mockingbird | 1960 |
| Harper Lee | Go Set a Watchman | 2015 |
<br>

This can be problematic for relational databases, which are designed to store data efficiently and avoid redundancy. To address this issue, we will separate the author and book information into two tables: authors and books. This approach eliminates duplicate data, ensuring data integrity and optimising storage.
<br>

| author_id | author_name |
| --- | --- |
| 1 | Arthur Conan Doyle |
| 2 | J.R.R. Tolkien |
| 3 | Harper Lee |
<br>

| book_id | book_title | year_published | author_id |
|---|---|---|---|
| 1 | The Adventures of Sherlock Holmes | 1887 | 1 |
| 2 | The Hobbit | 1937 | 2 |
| 3 | The Lord of the Rings | 1954 | 2 |
| 4 | To Kill a Mockingbird | 1960 | 3 |
| 5 | Go Set a Watchman | 2015 | 3 |

---
## 3.&nbsp; Creating the authors table with python 🐍
Let's start by creating the original DataFrame, including the repeated data.

In [ ]:
names = ["Arthur Conan Doyle", "J.R.R. Tolkien", "J.R.R. Tolkien", "Harper Lee", "Harper Lee"]
titles = ["The Adventures of Sherlock Holmes", "The Hobbit", "The Lord of the Rings", "To Kill a Mockingbird", "Go Set a Watchman"]
years = [1887, 1937, 1954, 1960, 2015]

non_relational_df = pd.DataFrame({"author_name": names,
                                  "book_title": titles,
                                  "year_published": years})

non_relational_df

,author_name,book_title,year_published
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887
1,J.R.R. Tolkien,The Hobbit,1937
2,J.R.R. Tolkien,The Lord of the Rings,1954
3,Harper Lee,To Kill a Mockingbird,1960
4,Harper Lee,Go Set a Watchman,2015


Now, let's select only the authors without any duplicates.

In [ ]:
authors_unique = non_relational_df["author_name"].unique()

authors_df = pd.DataFrame({"author_name": authors_unique})

authors_df

,author_name
0,Arthur Conan Doyle
1,J.R.R. Tolkien
2,Harper Lee


Fantastic! This DataFrame will be the foundation of our authors table.

---
## 4.&nbsp; Creating the matching authors table with SQL 💻

Ok, now we're ready to store this DataFrame in SQL. Before we can send the information in SQL, we need to make a table that has the same columns and data types to recieve the data. While we are creating a table for authors, we can also create the books table too.

Open MySQL Workbench, open a local connection, and open a new file. Then copy and paste the code from below.

```sql
-- Drop the database if it already exists
DROP DATABASE IF EXISTS sql_workshop ;

-- Create the database
CREATE DATABASE sql_workshop;

-- Use the database
USE sql_workshop;

-- Create the 'authors' table
CREATE TABLE authors (
    author_id INT AUTO_INCREMENT, -- Automatically generated ID for each author
    author_name VARCHAR(255) NOT NULL, -- Name of the author
    PRIMARY KEY (author_id) -- Primary key to uniquely identify each author
);

-- Create the 'books' table
CREATE TABLE books (
    book_id INT AUTO_INCREMENT, -- Automatically generated ID for each book
    book_title VARCHAR(255) NOT NULL, -- Title of the book
    year_published INT, -- Year the book was published
    author_id INT, -- ID of the author who wrote the book
    PRIMARY KEY (book_id), -- Primary key to uniquely identify each book
    FOREIGN KEY (author_id) REFERENCES authors(author_id) -- Foreign key to connect each book to its author
);
```

To download the sql file that we will follow for this section, [click here](https://drive.google.com/uc?export=download&id=1tln_33FM7D9wLckzxacBJNcMYqtyxybE)

If you'd like more information about MySQL data types [click here](https://www.w3schools.com/mysql/mysql_datatypes.asp).

---
## 5.&nbsp; Sending the information from this notebook to sql 📠
To establish a connection with the SQL database, we need to provide the notebook with the necessary information, which we do using the connection string below. You will need to modify only the password variable, which should match the password you set during MySQL Workbench installation.

In [ ]:
schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = "YOUR_PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

To send information to our sql databse we use the pandas method `.to_sql()`. The argument `if_exists="append"` says that we don't want to overwrite any existing data, but add on to what is already there.

In [ ]:
authors_df.to_sql('authors',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

Now, have a look at the table `authors` in MySQL Workbench, you should see that the names of the authors have appeared.

---
## 6.&nbsp; Retrieving information from sql to this notebook 📥
It's not only possible to send information to a SQL database, but also retrieve it too. Using `.read_sql()` in combination with the `connection_string` we can access the required data.

In [ ]:
authors_from_sql = pd.read_sql("authors", con=connection_string)
authors_from_sql

,author_id,author_name
0,1,Arthur Conan Doyle
1,2,J.R.R. Tolkien
2,3,Harper Lee


Using this same method, we can also perform SQL queries to only bring back certain sections of information instead of the whole DataFrame.

In [ ]:
pd.read_sql("""
            SELECT DISTINCT author_name
            FROM authors
            """,
            con=connection_string)

---
## 7.&nbsp; Preparing and sending the books table 📚
By extracting the authors table from our SQL database, we gain access to the unique identifier `author_id` assigned to each author. These `author_id`'s serve as pointers to their corresponding author records, allowing us to seamlessly link the `author_id`'s in the books table to their respective authors in the authors table, thereby completing the books table.

In [ ]:
books_df = non_relational_df.merge(authors_from_sql,
                                   on = "author_name",
                                   how="left")

books_df

,author_name,book_title,year_published,author_id
0,Arthur Conan Doyle,The Adventures of Sherlock Holmes,1887,1
1,J.R.R. Tolkien,The Hobbit,1937,2
2,J.R.R. Tolkien,The Lord of the Rings,1954,2
3,Harper Lee,To Kill a Mockingbird,1960,3
4,Harper Lee,Go Set a Watchman,2015,3


In [ ]:
books_df = books_df.drop(columns=["author_name"])

books_df

,book_title,year_published,author_id
0,The Adventures of Sherlock Holmes,1887,1
1,The Hobbit,1937,2
2,The Lord of the Rings,1954,2
3,To Kill a Mockingbird,1960,3
4,Go Set a Watchman,2015,3


In [ ]:
books_df.to_sql('books',
                if_exists='append',
                con=connection_string,
                index=False)

5

In [ ]:
books_from_sql = pd.read_sql("books", con=connection_string)
books_from_sql

,book_id,book_title,year_published,author_id
0,1,The Adventures of Sherlock Holmes,1887,1
1,2,The Hobbit,1937,2
2,3,The Lord of the Rings,1954,2
3,4,To Kill a Mockingbird,1960,3
4,5,Go Set a Watchman,2015,3


---
## 8.&nbsp; Challenge 😃
Now that you've learnt how to send and retrieve information, it's your turn to show off your skills. Create multiple tables in SQL for the data you scrapped about cities from Wikipedia. One should just be a table about the cities, the others should be facts about the cities.

| city_id | city |
| --- | --- |
| 1 | Berlin |
| 2 | Hamburg |
| 3 | Munich |

<br>

| City ID | Population | Year Data Retrieved |
|---|---|---|
| 1 | 3,850,809 | 2024 |
| 2 | 1,945,532 | 2024 |
| 3 | 1,512,491 | 2024 |

> **Pro Tip:** Visualise your relational database with pen and paper before you start coding. This can help you to identify any potential problems or inconsistencies in your design, and it can also make the coding process more efficient.